In [1]:
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2
from util import *
import glob

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
import time

## Train SVC

In [2]:
# Get all file paths
def get_all_files(paths):
    files = []
    for p in paths:
        f = glob.glob(p)
        # If there are no image in the directory, print error
        if (len(f) == 0):
            print("PATH ERROR")
        files.extend(f)
    return files

In [3]:
# Vehicle directories
vehicle_paths = ['vehicles/GTI_Far/image*.png', 'vehicles/GTI_Left/image*.png', 'vehicles/GTI_MiddleClose/image*.png', \
         'vehicles/GTI_Right/image*.png', 'vehicles/KITTI_extracted/*.png']
# Non-vehicle directories
non_vehicle_paths = ['non-vehicles/Extras/extra*.png', 'non-vehicles/GTI/image*.png']

In [4]:
vehicle_files = get_all_files(vehicle_paths)
non_vehicle_files = get_all_files(non_vehicle_paths)

In [5]:
color_space = 'HLS' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block

hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16) # Spatial binning dimensions
hist_bins = 16    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off

# Extract features of all the vehicles
vehicle_features = extract_features(vehicle_files, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
# Extract features of all the non-vehicles
non_vehicle_features = extract_features(non_vehicle_files, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
X = np.vstack((vehicle_features, non_vehicle_features)).astype(np.float64)

In [6]:
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(vehicle_features)), np.zeros(len(non_vehicle_features))))

# Split up data into randomized training and test sets
rand_state = 464
# Split data into 80% training data and 20% testing data
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))

Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 6108


In [7]:
# Use a linear SVC
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')

# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))

37.38 Seconds to train SVC...
Test Accuracy of SVC =  0.9911


## Find cars using sliding window

In [8]:
# Parameters for sliding window
ystart = 400
ystop = 656
scale = 1.5

In [9]:
from scipy.ndimage.measurements import label
import copy

In [10]:
image = mpimg.imread('test_images/test1.jpg')
# Store the heat map and the parameters for heat map
class Wrapper:
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    decay = 1 # 1 means no decay
    threshold = 1

In [11]:
# Identify bounding boxes and draw on the image
def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

In [12]:
# Use sliding window and extract features to finding bounding boxes for cars
def process_image(img):
    draw_img = np.copy(img)
    img = img.astype(np.float32)/255
    
    # Extract only area of interest in image
    img_tosearch = img[ystart:ystop,:,:]
    # The image was trained using HLS, therefore we convert to HLS
    ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2HLS)
    # Scale up the image
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
    
    # Color channels to get hog features from
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]

    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // pix_per_cell)-1
    nyblocks = (ch1.shape[0] // pix_per_cell)-1 
    nfeat_per_block = orient*cell_per_block**2
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell)-1 
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
    # Compute individual channel HOG features for the entire image
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    Wrapper.heat *= Wrapper.decay
    
    # Apply sliding window
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
          
            # Get color features
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)

            # Scale features and make a prediction
            test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))
            test_prediction = svc.predict(test_features)
            
            # Image patch predicted as car
            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                # Add bounding box to heat map
                Wrapper.heat[ytop_draw+ystart:ytop_draw+win_draw+ystart, xbox_left:xbox_left+win_draw] += 1
    
    # Apply threshold to heat map to remove noise and get the bounding boxes
    thresh_heat = copy.copy(Wrapper.heat)
    thresh_heat[thresh_heat <= Wrapper.threshold] = 0
    # Find final boxes from heatmap using label function
    labels = label(thresh_heat)
    draw_labeled_bboxes(draw_img, labels)
                
    return draw_img

In [13]:
image_dirs = glob.glob('test_images/test*.jpg')
# Iterate through all the test images
for image_dir in image_dirs:
    image = mpimg.imread(image_dir)
    
    # Reset heat image
    Wrapper.heat = np.zeros_like(image[:,:,0]).astype(np.float)
    Wrapper.threshold = 1
    out_img = process_image(image)
    
    mpimg.imsave('output_images/' + image_dir.split('\\')[1], out_img)

In [14]:
from moviepy.editor import VideoFileClip

In [15]:
# Read the video and run the 'process_image' function to all the frames
# Reset heat image
Wrapper.heat = np.zeros_like(image[:,:,0]).astype(np.float)
Wrapper.threshold = 5
Wrapper.decay = 5/6
output_file = 'project_video_out.mp4'
clip = VideoFileClip('project_video.mp4')
project_clip = clip.fl_image(process_image)
%time project_clip.write_videofile(output_file, audio=False)

[MoviePy] >>>> Building video project_video_out.mp4
[MoviePy] Writing video project_video_out.mp4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 1260/1261 [12:59<00:00,  1.32it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_out.mp4 

Wall time: 13min 1s
